# Llama Index Comparison

[Instructions](https://docs.llamaindex.ai/en/stable/examples/cookbooks/llama3_cookbook/)

[Prompts Editing](https://docs.llamaindex.ai/en/stable/examples/cookbooks/llama3_cookbook/)

In [35]:
!pip install -q llama-index llama-index-llms-huggingface llama-index-embeddings-huggingface chromadb llama-index-vector-stores-chroma


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [53]:
import torch
import chromadb
from transformers import AutoTokenizer
from llama_index.llms.huggingface import HugginqgFaceLLM
from llama_index.core.llms import ChatMessage
from IPython.display import Markdown, display


base_llama3 = '/root/llama/models--meta-llama--Meta-Llama-2-8B-Instruct/snapshots/e5e23bbe8e749ef0efcf16cad411a7d23bd23298/'
tunned_llama3 = "/root/llama/outputs/final_merged_checkpoint"

tokenizer = AutoTokenizer.from_pretrained(base_llama3)

stopping_ids = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>"),
]



In [7]:
# Optional quantization to 4bit
# import torch
# from transformers import BitsAndBytesConfig

# quantization_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_compute_dtype=torch.float16,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_use_double_quant=True,
# )

llm = HuggingFaceLLM(
    model_name=base_llama3,
    model_kwargs={
        "torch_dtype": torch.bfloat16,  # comment this line and uncomment below to use 4bit
        # "quantization_config": quantization_config
    },
    generate_kwargs={
        "do_sample": True,
        "temperature": 1.0,
        "top_p": 0.9,
    },
    tokenizer_name=base_llama3,
    stopping_ids=stopping_ids,
)


Loading checkpoint shards: 100%|██████████| 4/4 [00:48<00:00, 12.23s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [29]:
response = llm.complete("Who are the most important people in Hamas and what their role is? When answering use a list format")

print(response)

# messages = [
#     ChatMessage(role="system", content="You are CEO of MetaAI"),
#     ChatMessage(role="user", content="Introduce Llama3 to the world."),
# ]

# response = llm.chat(messages)

# print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


 and provide a isketch of the current leadership structure.
**Note:** Hamas' leadership structure is not publicly known, and its members are not always publicly identified. The following information is based on reports and sources and is subject to change.

1. **Ismail Haniyeh**:
	* Current leader of Hamas (2017-present)
	* Prime Minister of the Gaza Strip (2006-2014, 2014-present)
	* Has been involved in the organization since its founding in the 1980s
	* Known for his pragmatism and ability to navigate complex political situations
2. **Khaled Mashal**:
	* Former leader of Hamas (2004-2017)
	* Known for his hardline stance on Israel and his ability to mobilize international support for the organization
	* Exiled in Syria (2012-2019) due to security concerns
	* Currently resides in Qatar
3. **Fathi Hammad**:
	* Deputy leader of Hamas (2017-present)
	* Minister of Interior and National Security in the Gaza Strip (2007-2014)
	* Known for his involvement in the organization's military win

# RAG

In [55]:
from llama_index.core import PromptTemplate
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings, StorageContext
import chromadb


def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}<br>" f"**Text:** <br>"
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown("<br><br>"))


# load docs
documents = SimpleDirectoryReader(
    input_files=[f"RAG/{f}" for f in os.listdir('RAG/') if f.startswith('index')]
).load_data()


# embedding model
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

# Settings
Settings.embed_model = embed_model
Settings.llm = llm

# initialize client, setting path to save data
db = chromadb.PersistentClient(path="./RAG/chroma_db")

# create collection
chroma_collection = db.get_or_create_collection("news-rag-gaza")

# assign chroma as the vector_store to the context
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# create your index
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)


In [57]:
# load your index from stored vectors
index = VectorStoreIndex.from_vector_store(
    vector_store, storage_context=storage_context
)

In [58]:
# prompts_dict = query_engine.get_prompts()
# display_prompt_dict(prompts_dict)


In [102]:
query_engine = index.as_query_engine(similarity_top_k=3)
# prompts_dict = query_engine.get_prompts()
# display_prompt_dict(prompts_dict)
query = "Who are the most important people in Hamas and what their role is? When answering use a list format"

# res = query_engine.retrieve(query)
# print(res[0].node.text)

response = query_engine.query(query)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [103]:
print(response)



 Based on the provided context, it is not possible to identify the most important people in Hamas. The context only mentions specific individuals taken as hostages or victims of the conflict, but does not provide information about the leadership or key figures within Hamas. 

Additional context is necessary to provide a more accurate answer. However, it can be noted that Hamas is a Palestinian Sunni-Islamic fundamentalist organization that has been involved in the Israeli-Palestinian conflict and has been designated as a terrorist organization by several countries. As such, the leadership and key figures within Hamas are typically not publicly disclosed for security and political reasons. 

If additional context is provided, such as information about the current or past leaders of Hamas, their roles, or any specific individuals who hold significant power or influence within the organization, it would be possible to provide a more accurate answer.  Please provide more context. 

Please 

In [104]:
# query_engine.retrieve("hamas leaders")
# query_engine.query("What did paul graham do growing up?")

